In [9]:
import torch
import torchvision.transforms.functional as T
import torch.nn.functional as F
import numpy as np

from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
import sys
from pathlib import Path
import json
from functools import partial
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from avalanche.training.plugins import EvaluationPlugin
from avalanche.benchmarks import SplitTinyImageNet

# Get the parent directory of the current notebook
parent_dir = Path.cwd().parent

# Add the parent directory (which contains the svcca directory) to the Python path
sys.path.append(str(parent_dir))

from analysis.args import (
    MODEL_DIR,
    LAYERS,
    ALGOS,
    # LOAD_EXPERIENCES,
    NUM_EXPERIENCES_TRAINED_ON,
    TRUNCATE
)

from utils import (
    generate_experiences,
    adapt_model,
    model_loader,
    strategy_builder,
    StrategiesEnum,
    DataStreamEnum
)

from training.args import lwf_args, mas_args, naive_args
from training.models.models import MultiHeadVGGSmall


In [10]:
from typing import Any

class ModelWrapper: 
    def __init__(self, model, task_labels):
        self.model = model
        self.task_labels = task_labels
        
    def __getattr__(self, attr):
        if attr == "forward":
            return self.forward_wrapper
        return getattr(self.model, attr)

    def __call__(self, x):
        return self.forward_wrapper(x)
    
    def forward_wrapper(self, x):
        return self.model.forward(x, self.task_labels)

In [11]:
SPLIT = "test"
SAVE_CAM_DIR = "test_cams" if SPLIT == "test" else "train_cams"
LOAD_EXPERIENCES = list(range(9,10))
EVALUATE_EXPERIENCES = list(range(10))

In [12]:
def make_cam_dict(model, inputs, name, model_id, exp_id):
    cam_extractor = GradCAM(model, target_layer=LAYERS)
    out = model(inputs) 
    cam = cam_extractor(list(out.argmax(dim=1)), out)     
    cam = [layer_cam.tolist() for layer_cam in cam]
    cam_dict = dict(zip(LAYERS, cam))
    with open(f'../{SAVE_CAM_DIR}/{name}_cam_on_model_{model_id}_exp_{exp_id}.json', 'w') as f:
        json.dump(cam_dict, f)

In [13]:
# Split benchmark
benchmark = SplitTinyImageNet(
    NUM_EXPERIENCES_TRAINED_ON, return_task_id=True, dataset_root=None)

# Generate list of experiences
if (DataStreamEnum[SPLIT] == DataStreamEnum.train):
    stream = benchmark.train_stream
elif (DataStreamEnum[SPLIT] == DataStreamEnum.test):
    stream = benchmark.test_stream

experiences = generate_experiences(
    stream=stream, n_experiences=benchmark.n_experiences)

# Directory where all models are saved
lwf_dir = Path(f"../{MODEL_DIR}/lwf_150_saved_models")
mas_dir = Path(f"../{MODEL_DIR}/mas_150_saved_models")
naive_dir = Path(f"../{MODEL_DIR}/naive_150_saved_models")

criterion = CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# Base model
base_model = MultiHeadVGGSmall(n_classes=200)

# Iterate through all experiences to load
for load_experience in LOAD_EXPERIENCES:

    # Adapt base model with experiences
    if (load_experience > 0):
        adapt_model(base_model, experiences)

    # Load trained model paths
    pattern = f"*{load_experience}"
    lwf_path = list(lwf_dir.glob(pattern))[0]
    mas_path = list(mas_dir.glob(pattern))[0]
    naive_path = list(naive_dir.glob(pattern))[0]

    # Instantiate trained models
    lwf_model_partial = partial(ModelWrapper, model=model_loader(base_model=base_model, path=lwf_path))
    mas_model_partial = partial(ModelWrapper, model=model_loader(base_model=base_model, path=mas_path))
    naive_model_partial = partial(ModelWrapper, model=model_loader(base_model=base_model, path=naive_path))

    # Evaluation experiences
    for eval_exp in EVALUATE_EXPERIENCES:
        
        exp = experiences[eval_exp]
        labels = exp.classes_in_this_experience
        targets = [sample[1] for sample in exp.dataset]
        sample_indices = [targets.index(label) for label in labels]
        subset = exp.dataset.subset(sample_indices)
        dataloader = DataLoader(subset, batch_size=100)
        
        inputs, _, _ = next(iter(dataloader))
        
        # LWF
        names = ["lwf", "mas", "naive"]
        models = [lwf_model_partial(task_labels=exp.task_label), mas_model_partial(task_labels=exp.task_label), naive_model_partial(task_labels=exp.task_label)]
        
        for i in range(len(models)):
            make_cam_dict(model=models[i], inputs=inputs, name=names[i], model_id=load_experience, exp_id=eval_exp)
        